In [0]:
# Ingestão de Types - Magic: The Gathering (Versão Corrigida)
# Objetivo: Ingerir dados de types da API do Magic: The Gathering para staging em Parquet no S3
# Características: Dados brutos, formato Parquet, tabela de referência, particionamento, incremental

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import requests
import json
import logging
from datetime import datetime, timedelta
from pyspark.sql.functions import current_timestamp, lit, col, year, month, when
from pyspark.sql.types import *
import time

# =============================================================================
# CONFIGURAÇÃO DE SEGREDOS
# =============================================================================

def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

# =============================================================================
# CONFIGURAÇÕES GLOBAIS
# =============================================================================

# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configurações da API
API_BASE_URL = get_secret("api_base_url")
BATCH_SIZE = int(get_secret("batch_size", "100"))
MAX_RETRIES = int(get_secret("max_retries", "3"))

# Configurações do S3
S3_BUCKET = get_secret("s3_bucket")
S3_PREFIX = get_secret("s3_prefix")
S3_BASE_PATH = f"s3://{S3_BUCKET}/{S3_PREFIX}"

# Log das configurações
print("=" * 60)
print("CONFIGURAÇÕES PARA INGESTÃO DE TYPES")
print("=" * 60)
print("API_BASE_URL: [CONFIGURADO]")
print("S3_BASE_PATH: [CONFIGURADO]")
print("Tabela de referência - sem filtro temporal")
print("=" * 60)



In [0]:
# =============================================================================
# FUNÇÕES UTILITÁRIAS
# =============================================================================

def setup_s3_storage():
    try:
        # Verificar se o diretório existe (criar se necessário)
        try:
            dbutils.fs.ls(S3_BASE_PATH)
            print("Diretório do S3 já existe")
        except:
            dbutils.fs.mkdirs(S3_BASE_PATH)
            print("Diretório do S3 criado com sucesso")
        
        return True
        
    except Exception as e:
        print(f"Erro ao configurar S3 storage: {e}")
        return False

def make_api_request(endpoint, params=None, retries=MAX_RETRIES):
    url = f"{API_BASE_URL}/{endpoint}"
    print(f"Fazendo requisição para: {url}")
    
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, timeout=30)
            print(f"Status Code: {response.status_code}")
            
            if response.status_code == 200:
                data = response.json()
                print(f"Dados recebidos: {type(data)}")
                if isinstance(data, dict):
                    print(f"Chaves disponíveis: {list(data.keys())}")
                return data
            elif response.status_code == 429:  # Rate limit
                wait_time = min((attempt + 1) * 5, 60)
                print(f"Rate limit atingido. Aguardando {wait_time}s...")
                time.sleep(wait_time)
            elif response.status_code == 503:  # Service unavailable
                wait_time = min((attempt + 1) * 10, 120)
                print(f"Serviço indisponível. Aguardando {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"Erro {response.status_code} na API: {response.text[:200]}")
                if attempt < retries - 1:
                    time.sleep(5)
                
        except requests.exceptions.Timeout:
            print(f"Timeout na tentativa {attempt + 1}")
            if attempt < retries - 1:
                time.sleep(10)
        except requests.exceptions.RequestException as e:
            if attempt == retries - 1:
                print(f"Erro na requisição para endpoint após {retries} tentativas: {e}")
                return None
            print(f"Tentativa {attempt + 1} falhou, tentando novamente...")
            time.sleep(1)
        except json.JSONDecodeError as e:
            print(f"Erro ao decodificar JSON na tentativa {attempt + 1}: {e}")
            if attempt == retries - 1:
                return None
            time.sleep(1)
    
    return None

def clean_types_data(data):
    # data é uma lista de strings
    cleaned_data = []
    for item in data:
        if isinstance(item, str):
            cleaned_data.append({"type_name": item})
    return cleaned_data

def save_to_parquet(data, table_name):
    if not data:
        print(f"Nenhum dado para salvar na tabela {table_name}")
        return None
    try:
        if table_name == 'types':
            schema_fields = [StructField("type_name", StringType(), True)]
            schema = StructType(schema_fields)
            df = spark.createDataFrame(data, schema)
        else:
            df = spark.createDataFrame(data)
        df = df.withColumn("ingestion_timestamp", current_timestamp()) \
               .withColumn("source", lit("mtg_api")) \
               .withColumn("endpoint", lit(table_name))
        df = df.withColumn("partition_year", lit(datetime.now().year)) \
               .withColumn("partition_month", lit(datetime.now().month))
        partition_combinations = df.select("partition_year", "partition_month").distinct().collect()
        for partition_row in partition_combinations:
            partition_year = partition_row["partition_year"]
            partition_month = partition_row["partition_month"]
            partition_df = df.filter((col("partition_year") == partition_year) & 
                                   (col("partition_month") == partition_month))
            file_name = f"{partition_year}_{partition_month:02d}_{table_name}.parquet"
            file_path = f"{S3_BASE_PATH}/{file_name}"
            try:
                existing_files = dbutils.fs.ls(file_path)
                if len(existing_files) > 0:
                    print(f"Arquivo {file_name} já existe - pulando")
                    continue
            except:
                pass
            partition_df.drop("partition_year", "partition_month").write.mode("overwrite").format("parquet").save(file_path)
            print(f"Arquivo {file_name} criado com sucesso")
        print(f"Registros salvos como Parquet para {table_name}")
        return df
    except Exception as e:
        print(f"Erro ao salvar dados em {table_name}: {e}")
        return None

def ingest_simple_data(endpoint, table_name, data_key=None):
    print(f"Iniciando ingestão simples: {table_name}")
    
    if data_key is None:
        data_key = table_name
    
    data = make_api_request(endpoint)
    
    if data and data_key in data:
        table_data = data[data_key]
        print(f"Dados obtidos para {table_name}: {len(table_data)} registros")
        
        # Limpar dados se for types
        if table_name == 'types':
            print("Limpando dados de types...")
            table_data = clean_types_data(table_data)
            print(f"Dados limpos: {len(table_data)} registros")
        
        # Verificar se há dados antes de tentar salvar
        if not table_data:
            print(f"Nenhum dado válido para {table_name}")
            return None
        
        try:
            df = save_to_parquet(table_data, table_name)
            
            if df:
                count = df.count()
                print(f"{table_name}: {count} registros processados")
                display(df.limit(5))
            return df
        except Exception as e:
            print(f"Erro ao processar dados de {table_name}: {e}")
            return None
    else:
        print(f"Falha ao obter dados de {table_name}")
        if data:
            print(f"Chaves disponíveis nos dados: {list(data.keys())}")
        return None

# Verificar Spark
try:
    spark
    print("Spark disponível")
except NameError:
    print("Spark não está disponível - tentando obter do contexto")
    try:
        from pyspark.sql import SparkSession
        spark = SparkSession.builder.getOrCreate()
        print("Spark criado com sucesso")
    except Exception as e:
        print(f"Erro ao criar Spark: {e}")
        raise Exception("Spark não está disponível")

# Configurar S3 Storage
setup_success = setup_s3_storage()
if not setup_success:
    raise Exception("Falha ao configurar S3 storage")

print("Setup concluído com sucesso")



In [0]:
# Iniciar ingestão de types
print("Iniciando ingestão de types...")

types_df = ingest_simple_data(
    endpoint="types",
    table_name="types"
)

# Gerar relatório
print("=" * 50)
print("RELATÓRIO DE INGESTÃO DE TYPES")
print("=" * 50)

if types_df:
    print("Arquivos salvos.")
else:
    print("Falha na ingestão de types")

print("=" * 50)